In [9]:
!pip install mlflow==2.13.2 sagemaker-mlflow==0.1.0 cloudpickle==2.2.1

In [10]:
import boto3
import sagemaker
import sagemaker.session
import os
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.workflow.steps import TrainingStep
from sagemaker.sklearn.model import SKLearnModel
from sagemaker.workflow.model_step import ModelStep
from sagemaker.transformer import Transformer
from sagemaker.workflow.steps import TransformStep
from sagemaker.inputs import TransformInput
from sagemaker.workflow.pipeline import Pipeline
from sagemaker import get_execution_role
from sagemaker.workflow.parameters import ParameterString, ParameterInteger

from scripts.functions import *

import os
import pandas as pd
import numpy as np
import mlflow
from mlflow.tracking import MlflowClient

settings = read_settings('scripts/settings.json')

In [11]:
region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()
default_bucket = sagemaker_session.default_bucket()

pipeline_session = PipelineSession()

In [12]:
input_path = ParameterString(name="InputPath", default_value="")
output_path = ParameterString(name="OutputPath", default_value="")

In [13]:
transformer = Transformer(
    model_name="pipelines-6i7hk20pbxwo-01-churn-model-creat-XupKjmb2SN",
    instance_type="ml.m5.large",
    instance_count=1,
    output_path=output_path,
    accept="text/csv"
)

In [14]:
step_transform = TransformStep(
    name=settings["transformer_step_name"],
    transformer=transformer,
    inputs=TransformInput(
        data=input_path,
        content_type='text/csv', # It is neccessary because csv is not default format
        split_type='Line' # Each line equals one observation)
))

In [15]:
pipeline_name = f"01-churn-inference"
pipeline = Pipeline(
    name=pipeline_name,
    steps=[step_transform],
    parameters=[input_path, output_path]
)

In [16]:
pipeline.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:eu-west-1:211125740051:pipeline/01-churn-inference',
 'ResponseMetadata': {'RequestId': '163cf399-ff24-4541-b24e-0b255ffcb03a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '163cf399-ff24-4541-b24e-0b255ffcb03a',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '86',
   'date': 'Mon, 15 Jul 2024 14:38:51 GMT'},
  'RetryAttempts': 0}}